# 81 allow headcounts per process
troubleshooting the unenforced max constraint

- not finding the max of the process headcount and cell headcount
- always goes to cell headcount
- throws "solution inconsistency" warning

In [6]:
import numpy as np
from gpkit import Model, Variable
from gpx.primitives import Process

In [33]:
# set up processes times
t1 = Variable('t1', 100, 'min')
t2 = Variable('t2', 250, 'min')

# headcounts
h1 = Variable('h1', 2)
h2 = Variable('h2', 3)

# workstation headcount
hc = Variable('hc', 1)

In [34]:
# create list of tuples
proclist = [
    (t1, h1),
    (t2, h2)
]

In [35]:
class FindMax(Model):
    def setup(self, processes, hc):
        # total manhours
        self.tot = Variable('total effort', 'min')

        constr = []
        sums = []

        variables = []

        # generate the max headcount var
        for t,h in processes:
            # create a new maxvar
            maxvar = Variable('max at process: "{}"'.format(t.key.str_without()))

            # find the max
            constr.append(maxvar >= h)      # process hc
            constr.append(maxvar >= hc)     # cell hc

            # add to the sum
            sums.append(maxvar*t)

            # vars
            variables.extend([t,h])

        # sum to find total time
        constr.append(self.tot >= np.sum(sums))

        return constr


In [36]:
m1 = FindMax(proclist, hc)

In [37]:
m1.cost = m1.tot

In [38]:
print(m1)

FindMax7

Cost Function
-------------
 total effort

Constraints
-----------
 max at process: "t1" ≥ h1
 max at process: "t1" ≥ hc
 max at process: "t2" ≥ h2
 max at process: "t2" ≥ hc
 total effort ≥ max at process: "t1"·t1 + max at process: "t2"·t2


In [39]:
m1.substitutions

{h1: 2, hc: 1, h2: 3, t1: 100, t2: 250}

In [40]:
print(m1.solve().table())

Using solver 'cvxopt'
 for 3 free variables
  in 6 posynomial inequalities.
Solving took 0.275 seconds.

          ┃┓              ┓                      ┓
          ┃┃              ┃                      ┃
          ┃┃              ┃                      ┃
     Cost╺┫┃              ┣╸max at process: "t2" ┣╸h2
 (950min) ┃┣╸total effort ┃ (3)                  ┃ (3, fixed)
          ┃┃ (950min)     ┛                      ┛
          ┃┃              ┣╸max at process: "t1" ┣╸h1
          ┃┛              ┛ (2)                  ┛ (2, fixed)



       ┃┓          ┓
       ┃┃          ┃
       ┃┃          ┣╸total effort ≥ max at process: "t1"·t1 + max at process: "t2"·t2
       ┃┃          ┃
       ┃┃          ┛
       ┃┣╸FindMax7 ┓
       ┃┃          ┃
       ┃┃          ┣╸max at process: "t2" ≥ h2
       ┃┃          ┛
 Model╺┫┛          ┣╸max at process: "t1" ≥ h1
       ┃┓
       ┃┃
       ┃┣╸h2 = 3
       ┃┛
       ┃┓
       ┃┃
       ┃┣╸t2 = 250min
       ┃┛
       ┃┣╸h1 = 2
       ┃┣╸t1 